In [1]:
import numpy as np
import pickle
import os
import sys
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from numpy import genfromtxt

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Packages for modeling
from keras import models
from keras import layers
from keras import regularizers

Using TensorFlow backend.
/Users/kumarmohit/opt/anaconda3/envs/cil_project/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kumarmohit/opt/anaconda3/envs/cil_project/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kumarmohit/opt/anaconda3/envs/cil_project/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qi

In [2]:
GLOVE_DIRECTORY="glove.twitter.27B/"
GLOVE_FILE="glove.twitter.27B.200d.txt"
input_path = 'twitter-datasets/' 
ouput_path = 'output/'

# Reading the files and converting each word to embedding corresponding to the glove


In [3]:
positive_examples = list(open(input_path + "train_pos.txt", "r").readlines())
positive_examples = [s.strip() for s in positive_examples]  # -1000
negative_examples = list(open(input_path + "train_neg.txt", "r").readlines())
negative_examples = [s.strip() for s in negative_examples]
x = positive_examples + negative_examples
positive_labels = [1 for _ in positive_examples]
negative_labels = [0 for _ in negative_examples]
y = np.concatenate([positive_labels, negative_labels], 0)
shuffle_indices = np.random.permutation(np.arange(len(y)))
X_train= np.array(x)[shuffle_indices]
Y_train = np.array(y)[shuffle_indices]

In [38]:
# Loading glove embeddings from directory
embeddings = {}
with open(os.path.join(GLOVE_DIRECTORY, GLOVE_FILE), 'r',encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings[word] = coefs

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.1, random_state=37)
print('# Train data samples:', x_train.shape[0])
print('# Test data samples:', x_test.shape[0])
print('# Train data samples:', y_train.shape[0])
print('# Test data samples:', y_test.shape[0])

# Train data samples: 180000
# Test data samples: 20000
# Train data samples: 180000
# Test data samples: 20000


In [6]:
max([len(i) for i in x_test])

170

In [7]:
x_train[0]

'hahahaha subtwitters make laugh ! stupid dumbass !'

# Averaging the embedding for each sentence 

In [8]:
line_train_embeddings = []
for line in x_train:
    line_emb = np.zeros(200)
    fline_emb=[]
    length = len(line)
    for word in line.strip().split():
        tmp=[]
        if word in embeddings.keys():
            for (item1, item2) in zip(line_emb, embeddings[word]):
                tmp.append(item1+item2)
            line_emb=tmp        
    for item in line_emb:
        fline_emb.append(item/length)
    line_train_embeddings.append(fline_emb)


In [11]:
np.save(ouput_path + "train_embeddings.npy",line_train_embeddings)

In [12]:
line_test_embeddings = []
for line in x_test:
    line_emb = np.zeros(200)
    fline_emb=[]
    length = len(line)
    for word in line.strip().split():
        tmp=[]
        if word in embeddings.keys():
            for (item1, item2) in zip(line_emb, embeddings[word]):
                tmp.append(item1+item2)
            line_emb=tmp        
    for item in line_emb:
        fline_emb.append(item/length)
    line_test_embeddings.append(fline_emb)


In [13]:
np.save(ouput_path + "test_embeddings.npy",line_test_embeddings)

In [14]:
len(line_test_embeddings[6])

200

# Using Logistic Regression for the classification

In [16]:
scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=8, penalty='l2',max_iter=500)
model=scikit_log_reg.fit(line_train_embeddings,y_train)

[LibLinear]

In [17]:
score = model.score(line_test_embeddings, y_test)
print(score)

0.78125


In [108]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-5, random_state=42, max_iter=100)

In [109]:
model.fit(line_train_embeddings,y_train)

SGDClassifier(alpha=1e-05, max_iter=100, random_state=42)

In [110]:
score = model.score(line_test_embeddings, y_test)
print(score)

0.7813


# Using Random Forest Classifier 

In [ ]:
model = RandomForestClassifier(n_estimators=1000,max_depth=None,n_jobs=-1)

In [ ]:
model.fit(line_train_embeddings,y_train)

In [ ]:
score = model.score(line_test_embeddings, y_test)
print(score)

# Final predictions 

In [18]:
final_examples = list(open(input_path + "test_data.txt", "r").readlines())
final_test_embeddings = []
for line in final_examples:
    line_emb = np.zeros(200)
    fline_emb=[]
    length = len(line)
    for word in line.strip().split():
        tmp=[]
        if word in embeddings.keys():
            for (item1, item2) in zip(line_emb, embeddings[word]):
                tmp.append(item1+item2)
            line_emb=tmp        
    for item in line_emb:
        fline_emb.append(item/length)
    final_test_embeddings.append(fline_emb)

In [19]:
np.save(ouput_path + "final_test_embeddings.npy",final_test_embeddings)

In [113]:
len(final_test_embeddings[34])

200

In [42]:
sub=model.predict(final_test_embeddings)

In [43]:
len(sub)

10000

In [44]:
ans=[]
for index, row in enumerate(sub):
    if(row==0):
        ans.append(-1)
    else:
        ans.append(row)

In [27]:
import pandas as pd
def replace_com_tab(file1):
    idd=[]
    val=[]
    text_pos = open(file1, "r")
    for line in text_pos:
        currentline = line.split(",",1)
        #print(currentline[1])
        idd.append(int(currentline[0]))
        #tmp=currentline[1].replace(r'\n', ' ', regex=True)
        val.append(currentline[1])
    data={1 : val}
    #print(data)
    df = pd.DataFrame(data)
    df1= pd.DataFrame({
    'id':idd,
    'text': df[1].replace(r'\n', ' ', regex=True)
    })
    #return df
    #df1.to_csv('../data/test1.tsv', sep='\t',index=False)
    return idd

In [28]:
idd=replace_com_tab(input_path + 'test_data.txt')

In [30]:
len(idd)

10000

In [45]:
sav=pd.DataFrame({'Id':idd,'Prediction':ans})

In [46]:
sav.to_csv(ouput_path + 'RF.csv',index=False, header=True)

In [ ]:
# 0.77 score for logistic regression and 0.758 with random forest Classifier